## wps_climdex_rmm

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf)'s functions which return the annual count of days where daily precipitation is more than some threshold per day

- **Precipitation Exceeding 10mm Per Day** ([climdex.r10mm](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=16)): This function takes a climdexInput object as input and computes the climdex index R10mm:  theannual count of days where daily precipitation is more than 10mm per day.

- **Precipitation Exceeding 20mm Per Day** ([climdex.r20mm](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=17)): This function takes a climdexInput object as input and computes the climdex index R20mm:  theannual count of days where daily precipitation is more than 20mm per day.

- **Precipitation Exceeding A Specified Amount Per Day** ([climdex.rnnmm](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=20)): This function takes a climdexInput object as input and computes the climdex index Rnnmm:  theannual count of days where daily precipitation is more thannnmm per day

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url
from wps_tools.R import rda_to_vector, construct_r_out, test_rda_output

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_rmm?

Signature:
quail.climdex_rmm(
    climdex_input,
    threshold=None,
    loglevel='INFO',
    output_file='output.rda',
    output_formats=None,
)
Docstring:
The annual count of days where daily precipitation is more than [threshold] mm per day

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename to store the output Rdata (extension .rda)
threshold : float
    mm threshold for daily precipitation
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/github/quail/</tmp/quail-venv/lib/python3.8/site-packages/birdy/client/base.py-5>
Type:      method


#### Run wps_climdex_rmm with climdex.r10mm

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="r10mm_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_rmm(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            threshold=10.0,
            output_file=output_file.name,
        )
r10mm_url = output.get()[0]

You can also use rds input
#### Run wps_climdex_rmm with climdex.r20mm with rds input

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="r20mm_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_rmm(
            climdex_input=resource_filename("tests","data/climdexInput.rds"),
            threshold=20.0,
            output_file=output_file.name,
        )
r20mm_url = output.get()[0]

You can use multiple inputs
#### Run wps_climdex_rmm with climdex.rnnmm with rds input with multiple inputs

In [8]:
climdex_inputs = [
    resource_filename("tests","data/climdexInput.rds"),
    resource_filename("tests","data/climdexInput.rda"),
    resource_filename("tests","data/climdex_input_multiple.rda")
]
with NamedTemporaryFile(suffix=".rda", prefix="rm=nnmm_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_rmm(
            climdex_input=climdex_inputs,
            threshold=9.7,
            output_file=output_file.name,
        )
rnnmm_url = output.get()[0]

Access the output with **rda_to_vector** or **construct_r_out** from **wps_tools.R**

In [9]:
# NBVAL_IGNORE_OUTPUT
# use print() to see whole vector
r10mm = rda_to_vector(r10mm_url, "r10.0mm1_ci")
print(f"r10mm\n{r10mm}")
r20mm = rda_to_vector(r20mm_url, "r20.0mm1_ci")
print(f"r20mm\n{r20mm}")
rnnmm = rda_to_vector(rnnmm_url, "r9.7mm1_ci")
print(f"rnnmm\n{rnnmm}")

r10mm
1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 
  NA   24   NA   NA   NA   26   27   22   34   30   16   25   26   28   19   26 
1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 
  33   NA   NA   15   18   27   26   NA   33   NA   NA   NA   NA   NA   22   44 
1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 
  NA   22   NA   NA   31   NA   41   34   42   16   21   NA   35   NA 

r20mm
1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 
  NA    7   NA   NA   NA    6   10    9   13    7    4    2   12   14    3    6 
1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 
  11   NA   NA    2    7    8    8   NA    7   NA   NA   NA   NA   NA    9   11 
1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 
  NA    7   NA   NA   10   NA    9   13   13    2    7   NA   12   NA 

rnnmm
1959 1960 1961 1962 1963 1964 1965 1966 1967 196

In [10]:
# NBVAL_IGNORE_OUTPUT
construct_r_out([r10mm_url, r20mm_url, rnnmm_url])

[[R object with classes: ('numeric',) mapped to:
  [     nan, 24.000000,      nan,      nan, ..., 21.000000,      nan, 35.000000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 7.000000,      nan,      nan, ..., 7.000000,      nan, 12.000000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 24.000000,      nan,      nan, ..., 22.000000,      nan, 36.000000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 24.000000,      nan,      nan, ..., 22.000000,      nan, 36.000000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 24.000000,      nan,      nan, ..., 22.000000,      nan, 36.000000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 24.000000,      nan,      nan, ..., 22.000000,      nan, 36.000000,      nan]]]

In [11]:
test_rda_output(
    r10mm_url, "r10.0mm1_ci", "expected_rmm.rda", "expected_r10mm"
    )

test_rda_output(
    r20mm_url, "r20.0mm1_ci", "expected_rmm.rda", "expected_r20mm"
    )

test_rda_output(
    rnnmm_url, "r9.7mm1_ci", "expected_rmm.rda", "expected_rnnmm"
    )